In [ ]:
import numpy as np, math, re
from itertools import product

def read_file_lines(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        
        return [line.strip() for line in lines]

In [ ]:
test_input = [
    "0:", "###", "##.", "##.", "",
    "1:", "###", "##.", ".##", "",
    "2:", ".##", "###", "##.", "",
    "3:", "##.", "###", "##.", "",
    "4:", "###", "#..", "###", "",
    "5:", "###", ".#.", "###", "",
    "4x4: 0 0 0 0 2 0",
    "12x5: 1 0 1 0 2 2",
    "12x5: 1 0 1 0 3 2"
]

In [ ]:
def parse_input(text):
    if isinstance(text, list):
        text = "\n".join(text)

    shapes = []
    regions = []

    lines = iter(text.strip().splitlines())
    for raw in lines:
        line = raw.strip()
        if not line:
            continue

        if re.match(r'^\d+:\s*$', line):
            shape = []
            for _ in range(3):
                row = next(lines).strip()
                shape.append([1 if c == "#" else 0 for c in row])
            shapes.append(shape)

        elif ":" in line and not re.match(r'^\d+:\s*$', line):
            key, values = line.split(":")
            dims = [int(x) for x in key.strip().lower().split("x")]
            nums = [int(x) for x in values.strip().split()]
            regions.append([dims, nums])

    return shapes, regions

from copy import deepcopy

def rotate_90(arr):
    return [list(row) for row in zip(*arr[::-1])]

def flip_vertical(arr):
    return arr[::-1]

def flip_horizontal(arr):
    return [row[::-1] for row in arr]

def generate_orientations(shape):
    """Generate all unique rotations/flips of a shape."""
    orientations = []

    # original + 3 rotations
    current = shape
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # horizontal flip + rotations
    flipped_h = flip_horizontal(shape)
    current = flipped_h
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # vertical flip + rotations
    flipped_v = flip_vertical(shape)
    current = flipped_v
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # remove duplicates
    unique_orientations = []
    seen = set()
    for o in orientations:
        t = tuple(tuple(row) for row in o)
        if t not in seen:
            seen.add(t)
            unique_orientations.append(o)

    return unique_orientations

def place_shape_on_grid(grid, shape):
    """
    Place a single shape on every position + orientation on a grid.
    Returns a list of new grids where no cell exceeds 1.
    """
    rows = len(grid)
    cols = len(grid[0])
    placements = []

    orientations = generate_orientations(shape)

    for rot in orientations:
        s_rows = len(rot)
        s_cols = len(rot[0])

        for r in range(rows - s_rows + 1):
            for c in range(cols - s_cols + 1):
                new_grid = deepcopy(grid)
                overlap = False
                for i in range(s_rows):
                    for j in range(s_cols):
                        new_grid[r+i][c+j] += rot[i][j]
                        if new_grid[r+i][c+j] > 1:
                            overlap = True
                            break
                    if overlap:
                        break
                if not overlap:
                    placements.append(new_grid)

    return placements

def place_shapes_recursively(grid, list_of_shapes):
    """
    Recursively place each shape in list_of_shapes on the previous placement grids.
    Skip any grid that would cause an overlap (cell > 1).
    """
    if not list_of_shapes:
        return [grid]  # base case: no shapes left

    first_shape = list_of_shapes[0]
    rest_shapes = list_of_shapes[1:]

    # place the first shape on current grid
    first_placements = place_shape_on_grid(grid, first_shape)

    # recursively place remaining shapes on each valid placement
    all_grids = []
    for g in first_placements:
        all_grids.extend(place_shapes_recursively(g, rest_shapes))

    return all_grids
    
def process_regions(input):
    s, regions = parse_input(input)

    count = 0

    for region in regions:
        grid = [[0 for _ in range(region[0][0])] for _ in range(region[0][1])]
        r = region[1]

        list_of_shapes = []
        for i in range(0, len(r)):
            for j in range(0, r[i]):
                list_of_shapes.append(s[i])

        grids = place_shapes_recursively(grid, list_of_shapes)
            
        if len(grids) > 0:
            count += 1
    
    return count

process_regions(test_input)